In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from keras_tuner import HyperModel, GridSearch
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

In [ ]:
training_set = datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/Front On',
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

Found 326 images belonging to 3 classes.


In [ ]:
test_set = datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/Front On',
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

Found 80 images belonging to 3 classes.


In [ ]:
class VGGHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=self.input_shape)

        # Add custom layers
        x = Flatten()(base_model.output)
        x = Dense(units=hp.Choice('dense_units', [128, 192, 256, 320, 384, 448, 512]), activation='relu')(x)
        x = Dropout(rate=hp.Choice('dropout_rate', [0.3, 0.4, 0.5, 0.6, 0.7]))(x)
        predictions = Dense(self.num_classes, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        # Freeze the base model layers
        for layer in base_model.layers:
            layer.trainable = False

        # Compile the model
        model.compile(
            optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

In [ ]:
hypermodel = VGGHyperModel(input_shape=(256, 256, 3), num_classes=3)

In [ ]:
tuner = GridSearch(
    hypermodel,
    objective='val_accuracy',
    directory='vgg_hyperparameter_tuning',
    project_name='vgg16_grid_tuning'
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
tuner.search(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=10,
    validation_steps=80
)

Trial 70 Complete [00h 03m 44s]
val_accuracy: 0.38749998807907104

Best val_accuracy So Far: 0.48750001192092896
Total elapsed time: 04h 38m 45s


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"Best Dense Units: {best_hps.get('dense_units')}")
print(f"Best Dropout Rate: {best_hps.get('dropout_rate')}")
print(f"Best Optimizer: {best_hps.get('optimizer')}")

Best Dense Units: 256
Best Dropout Rate: 0.4
Best Optimizer: rmsprop


In [ ]:
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    training_set,
    steps_per_epoch=10,
    epochs=20,
    validation_data=test_set,
    validation_steps=80
)

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.4060 - loss: 16.6984 - val_accuracy: 0.4000 - val_loss: 5.6966
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.3325 - loss: 5.3994 - val_accuracy: 0.3500 - val_loss: 3.1489
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 721ms/step - accuracy: 0.5000 - loss: 3.4091 - val_accuracy: 0.3125 - val_loss: 2.0071
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.4485 - loss: 2.3139 - val_accuracy: 0.2625 - val_loss: 2.1125
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.3936 - loss: 1.3273 - val_accuracy: 0.4000 - val_loss: 1.1881
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 856ms/step - accuracy: 0.3750 - loss: 1.1150 - val_accuracy: 0.4125 - val_loss: 1.1192
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.5091 - loss: 0.9488 - val_accuracy: 0.3375 - val_loss: 1.1540
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.5002 - loss: 1.0191 - val_accuracy: 0.2875 - val_

In [ ]:
final_scores = best_model.evaluate(test_set)
print(f"Final Test Accuracy: {final_scores[1]}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.3990 - loss: 1.6749
Final Test Accuracy: 0.4000000059604645


In [ ]:
y_pred = best_model.predict(test_set)
y_true = test_set.labels
y_pred_classes = np.argmax(y_pred, axis=1)

5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step


In [ ]:
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.2645833333333333
Recall: 0.35
F1 Score: 0.3


In [ ]:
best_model.save('best_modelfrontonVGG.keras')